#### Zadanie 1.
Zadanie 1 polega na wyznaczeniu i wizualizacji przebiegu sygnału rzeczywistego oraz jego widma amplitudowego i fazowego, eksperementalnej weryfikacji twierdzenie Parsevala oraz porównania czasu wykonania algorytmu FFT z teoretycną zlożonością obliczeniową.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

N = 8
T = 0.5
dt = T / N
t = np.arange(N) * dt
s = np.sin(4 * np.pi * t)

plt.figure()
plt.stem(t, s)
plt.title('Sampled Signal s[n]')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

X = np.fft.fft(s)
freq = np.fft.fftfreq(N, dt)

plt.figure()
plt.stem(freq, np.abs(X))
plt.title('Amplitude Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.show()

plt.figure()
plt.stem(freq, np.angle(X))
plt.title('Phase Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase (rad)')
plt.show()

energy_time = np.sum(np.abs(s)**2)
energy_freq = (1/N) * np.sum(np.abs(X)**2)
print(f"Energy in time domain: {energy_time:.6f}")
print(f"Energy in frequency domain: {energy_freq:.6f}")

Ns = [2**l for l in range(1, 13)]
times = []
for N in Ns:
    x = np.random.rand(N)
    start = time.perf_counter()
    np.fft.fft(x)
    end = time.perf_counter()
    times.append(end - start)

plt.figure()
plt.plot(Ns, times)
plt.title('FFT Computation Time vs Number of Samples')
plt.xlabel('Number of Samples (N)')
plt.ylabel('Time (seconds)')
plt.show()

for N, t_elapsed in zip(Ns, times):
    print(f"N={N:5d}, FFT time={t_elapsed:.6e} s")


Otrzymujemy wyniki:

![](task1_plot1.png) ![](task1_plot2.png)
![](task1_plot3.png) ![](task1_plot4.png)

Energy in time domain: 4.000000

Energy in frequency domain: 4.000000

N=    2, FFT time=6.029999e-05 s

N=    4, FFT time=3.111000e-04 s

N=    8, FFT time=3.020001e-05 s

N=   16, FFT time=2.380001e-05 s

N=   32, FFT time=1.490001e-05 s

N=   64, FFT time=1.150000e-05 s

N=  128, FFT time=2.250000e-05 s

N=  256, FFT time=1.390000e-05 s

N=  512, FFT time=1.690000e-05 s

N= 1024, FFT time=4.090001e-05 s

N= 2048, FFT time=7.230000e-05 s

N= 4096, FFT time=7.929999e-05 s

Podsumując, widmo amplitudowe i fazowe podtwierdzają obecność jednej czystej składowej sinusoidalnej. Twierdzenie Parsevala jest spełnione (różnica < 1e-6). Wykres czasu FFT rośnie zgodnie z O(N log N).

#### Zadanie 2.
Zadanie 2 polegana na zbadaniu wpływu przesunięcia w czasie na postać widma amplitudowego i widma fazowego dyskretnego sygnału harmonicznego.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 2
N = 88
def s(n):
    return A * np.cos((2 * np.pi * n) / N)
n0_values = [0, N//4, N//2, 3*N//4]

for n0 in n0_values:
    signal = [s(n - int(n0)) for n in range(N)]
    X = np.fft.fft(signal)
    freq = np.fft.fftfreq(N, d=1)

    X[np.abs(X) < 1e-6] = 0

    plt.figure()
    plt.stem(freq, np.abs(X))
    plt.title(f'Amplitude Spectrum, n0 = {n0}')
    plt.xlabel('Normalized Frequency (cycles/sample)')
    plt.ylabel('Magnitude')
    plt.show()

    plt.figure()
    plt.stem(freq, np.angle(X).round(2))
    plt.title(f'Phase Spectrum, n0 = {n0}')
    plt.xlabel('Normalized Frequency (cycles/sample)')
    plt.ylabel('Phase (radians)')
    plt.show()

Otrzymujemy wyniki:

![](task2_plot1.png) ![](task2_plot2.png) ![](task2_plot3.png) ![](task2_plot4.png)
![](task2_plot5.png) ![](task2_plot6.png) ![](task2_plot7.png) ![](task2_plot8.png)

Widmo amplitudowe pozostaje niezmienione dla wszystkich przesunięć n0. Na każdym wykresie występują dwa symetryczne piko o amplitudzie trochę więcej niż 80 (dla dodatniej i ujemnej częstotliwości), co potwierdza, że czasowe przesunięcie sygnału nie wpływa na wartość amplitudy jego składowych częstotliwościowych. Faza zmienia się liniowo w zależności od n0, zgodnie z własnością przesunięcia w dziedzinie czasu (X[k] = -2pi*k*n0/N). Widzimy więc, że przesunięcie sugnału powoduje wyłacznie przesunięcie fazy, pozycje i wysokości pików amplitudowych pozostają takie same.

#### Zadanie 3.
Zadanie 3 polega na zbadaniu wpływu dopełnienia zerami na postać widma amplitudowego i widma fazowego dyskretnego
sygnału.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 4
N = 12
def s1(n):
    return A*((n % N) / N)
N0 = [0, N, 4*N, 9*N]

for n_val in N0:
    signal = [s1(n) for n in range(N)]
    M = len(signal) + n_val
    dft = np.fft.fft(signal, n=M)

    sig_amp = dft.copy()
    sig_amp[np.abs(dft) < 1e-16] = 0

    sig_faz = dft.copy()
    sig_faz[np.abs(dft) < 1e-8] = 0

    freq = np.fft.fftshift(np.fft.fftfreq(M, d=1))
    sig_amp = np.fft.fftshift(sig_amp)
    sig_faz = np.fft.fftshift(sig_faz)

    plt.figure()
    plt.stem(freq, np.abs(sig_amp))
    plt.title(f'Amplitude Spectrum, zero-pad = {n_val}')
    plt.xlabel('Normalized Frequency (cycles/sample)')
    plt.ylabel('Magnitude')
    plt.show()

    plt.figure()
    plt.stem(freq, np.angle(sig_faz))
    plt.title(f'Phase Spectrum, zero-pad = {n_val}')
    plt.xlabel('Normalized Frequency (cycles/sample)')
    plt.ylabel('Phase (radians)')
    plt.show()

Otrzymujemy wyniki:

![](task3_plot1.png) ![](task3_plot2.png) ![](task3_plot3.png) ![](task3_plot4.png)
![](task3_plot5.png) ![](task3_plot6.png) ![](task3_plot7.png) ![](task3_plot8.png)

Dopełnienie zerami nie wpływa na rzeczywiste wartości amplitudy ani fazy sygnału, niw zmienia położenia ani wysokości pików. Główną korzyścią zero-paddingu jest poprawa rozdzielności częstotliwościowej, co umożliwia dokładniejsze zobrazowanie kształtu widma.

#### Zadanie 4.
Zadanie 4 polega na wygenerowaniu dyskretnego sygnału będącego sumą trzech sinusoid.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a1, f1 = 0.2, 2000
a2, f2 = 0.5, 6000
a3, f3 = 0.6, 10000
fs = 48000

N_list = [2048, int(3/2 * 2048)]

for N in N_list:
    t = np.arange(N) / fs
    s = a1 * np.sin(2*np.pi*f1*t) + a2 * np.sin(2*np.pi*f2*t) + a3 * np.sin(2*np.pi*f3*t)

    X = np.fft.fft(s)
    PSD = (np.abs(X)**2) / N
    freq = np.fft.fftfreq(N, d=1/fs)

    freq_shift = np.fft.fftshift(freq)
    PSD_shift = np.fft.fftshift(PSD)

    plt.figure()
    plt.plot(freq_shift, PSD_shift)
    plt.title(f'Power Spectral Density (N = {N})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.xlim(0, fs/2)  # show positive frequencies only
    plt.show()

    print(f"Completed PSD plot for N = {N}")


Otrzymujemy wyniki:

![](task4_plot1.png) ![](task4_plot2.png)

Dla N = 2048 widoczne są trzy ostre piki przy 2000 Hz, 6000 Hz i 10000 Hz, ale dwa z nich (2000 Hz i 10000 Hz) sa lekko rozmyte (posiadają boczne lobów). To efekt przecieku widma, ponieważ te częstotliwości nie leżą dokładnie na siatce FFT (nie są całkowitą wielokrotnością df). Dla N = 3072 wszystkie trzy piki są wąskie i pozbawione bocznych lobów, brak przecieku widma. Wynika to z faktu, że przy N = 3072 df = 15.625 Hz, a 2000, 6000, 10000 Hz są dokładnie całkowitymi wielokrotnościami tej wartośi. Zmiana liczby próbek z 2048 na 3072 poprawia dopasowanie tonów do linii FFT i eliminuje przeciek widma. Oznacza to, że aby uniknąć przecieku, należy dobrać N tak, żeby interesujące częstotliwości trafiały dokładnie na dyskretne biny FFT.